In [ ]:
%pip install -r ../requirements.txt

In [1]:
base_dir: bool = False

In [2]:
import os, sys

if not base_dir:
    os.chdir('../')
    base_dir = True

import numpy as np, pandas as pd, matplotlib.pyplot as plt, sklearn

from sklearn.metrics import accuracy_score

from utils.data_tools import inbreast, cbis_ddsm, image_processing
from utils.models import models

In [3]:
inbreast.download()
cbis_ddsm.download()

Path already exists for INBreast!
Path already exists for CBIS-DDSM!


In [8]:
# Get training features from cbis_ddsm and validation data from inbreast

class cbis_ddsm_data:

    x_train, x_test, y_train, y_test = data = cbis_ddsm.get_images()

class inbreast_data:

    x_train, x_test, y_train, y_test = data = inbreast.get_images()

    x, y = np.concat([x_train, x_test]), np.concat([y_train, y_test])

409/409


In [30]:
# Create model wrapper

M = models(cbis_ddsm_data.data)

In [31]:
# Contruct and Train model
cnn = M.SmallCNN(epochs=3, lr=0.01, metrics=['auc', 'accuracy'])

Epoch 1/3
72/72 ━━━━━━━━━━━━━━━━━━━━ 13s 153ms/step - accuracy: 0.7988 - auc: 0.8467 - loss: 0.9398
Epoch 2/3
72/72 ━━━━━━━━━━━━━━━━━━━━ 11s 148ms/step - accuracy: 0.9798 - auc: 0.9928 - loss: 0.0860
Epoch 3/3
72/72 ━━━━━━━━━━━━━━━━━━━━ 11s 148ms/step - accuracy: 0.9813 - auc: 0.9908 - loss: 0.0890
18/18 ━━━━━━━━━━━━━━━━━━━━ 1s 47ms/step - accuracy: 0.9802 - auc: 0.9985 - loss: 0.0595

            SmallCNN stats

            BinaryCrossentropy Loss: 0.04648654907941818
            auc: 0.9989706873893738
accuracy: 0.9842657446861267
            


In [32]:
# Validate on INBreast
y_pred = [np.round(p) for p in cnn.predict(inbreast_data.x_test, verbose=0)]

print(inbreast_accuracy := accuracy_score(inbreast_data.y_test, y_pred))

0.9878048780487805


In [33]:
# Model information

from timeit import timeit

def predict_100_images() -> None:
    cnn.predict(cbis_ddsm_data.x_test[: 10], verbose=0)

print(f"""
Validation Accuracy (INBreast): {round(inbreast_accuracy, 2) * 100}%
Number of Params: {(params := cnn.count_params())}
Memory Allocation: {round((params * 4) / (1024**2), 2)} MB
Run time (100 images GPU): {timeit(predict_100_images, number=1):.2f} sec
""")


Validation Accuracy (INBreast): 99.0%
Number of Params: 161313
Memory Allocation: 0.62 MB
Run time (100 images GPU): 0.08 sec



In [35]:
cnn.save('models/SmallCNN.keras')

In [ ]:
# GradCAM implementation

# from utils.grad_cam import GradCAM_Tools

# GradCAM_Tools.display.GradCAM(cnn, inbreast_data.x[0])